## Develop a hand gesture recognition model that can accurately identify and classify different hand gestures from image or video data, enabling intuitive human-computer interaction and gesture-based control systems.

 # 1. Data Preparation

In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set image dimensions
IMG_SIZE = 128

def load_images_from_folder(folder_path):
    images = []
    labels = []
    for subject_folder in os.listdir(folder_path):
        subject_path = os.path.join(folder_path, subject_folder)
        if os.path.isdir(subject_path):
            for gesture_folder in os.listdir(subject_path):
                gesture_path = os.path.join(subject_path, gesture_folder)
                if os.path.isdir(gesture_path):
                    for img_file in os.listdir(gesture_path):
                        img_path = os.path.join(gesture_path, img_file)
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                        images.append(img)
                        labels.append(gesture_folder)  # Use gesture folder as the label
    return np.array(images), np.array(labels)

data_path = r'C:\Users\siddartha\Downloads\archive (1)\leapGestRecog\leapGestRecog'
images, labels = load_images_from_folder(data_path)

# Reshape and normalize
images = images.reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0


# 2. Label Encoding

In [2]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)  # Converts text labels to numeric


# 3. Train-Test Split

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)


# 4. Model Design

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_cnn_model(input_shape):
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten and Dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))  # Prevent overfitting
    model.add(layers.Dense(10, activation='softmax'))  # 10 gestures

    return model

cnn_model = build_cnn_model((IMG_SIZE, IMG_SIZE, 1))
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.summary()


C:\Users\siddartha\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,354 (12.61 MB)

 Trainable params: 3,305,354 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

# 5. Data Augmentation (Optional)

In [5]:
datagen = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
datagen.fit(X_train)


# 6. Train the Model

In [6]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

history = cnn_model.fit(datagen.flow(X_train, y_train, batch_size=32),
                        epochs=5, validation_data=(X_test, y_test),
                        callbacks=[early_stopping])


Epoch 1/5


C:\Users\siddartha\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


337/337 ━━━━━━━━━━━━━━━━━━━━ 92s 268ms/step - accuracy: 0.2448 - loss: 2.0549 - val_accuracy: 0.7854 - val_loss: 0.6779
Epoch 2/5
337/337 ━━━━━━━━━━━━━━━━━━━━ 89s 265ms/step - accuracy: 0.6483 - loss: 0.9989 - val_accuracy: 0.9093 - val_loss: 0.2877
Epoch 3/5
337/337 ━━━━━━━━━━━━━━━━━━━━ 94s 278ms/step - accuracy: 0.7642 - loss: 0.6773 - val_accuracy: 0.9535 - val_loss: 0.1623
Epoch 4/5
337/337 ━━━━━━━━━━━━━━━━━━━━ 92s 273ms/step - accuracy: 0.8542 - loss: 0.4231 - val_accuracy: 0.9866 - val_loss: 0.0837
Epoch 5/5
337/337 ━━━━━━━━━━━━━━━━━━━━ 90s 267ms/step - accuracy: 0.8874 - loss: 0.3327 - val_accuracy: 0.9825 - val_loss: 0.0720


# 7. Evaluate the Model

In [7]:
test_loss, test_acc = cnn_model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.2f}')


85/85 ━━━━━━━━━━━━━━━━━━━━ 6s 75ms/step - accuracy: 0.9821 - loss: 0.0710
Test accuracy: 0.98


# 8. Saving the Model

In [8]:
cnn_model.save('gesture_recognition_model.h5')


In [14]:
img_size = 224
gesture_classes = [
    '01_palm', '02_l', '03_fist', '04_fist_moved', 
    '05_thumb', '06_index', '07_ok', '08_palm_moved', 
    '09_c', '10_down'
]

# 9. Load the Model

In [17]:
from tensorflow.keras.models import load_model

# Load the trained model
loaded_model = load_model('gesture_recognition_model.h5')


# 10. Test on a Single Image

In [20]:
import cv2
import numpy as np

# Function to preprocess an image (resize, normalize)
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load image as grayscale
    img = cv2.resize(img, (128, 128))  # Resize image to match model input
    img = img / 255.0  # Normalize pixel values
    img = np.reshape(img, (1, 128, 128, 1))  # Reshape to (1, 128, 128, 1) for prediction
    return img

# Example image path (update this to the actual path of your test image)
test_img_path = r"C:\Users\siddartha\Downloads\archive (1)\leapGestRecog\leapGestRecog\09\10_down\frame_09_10_0164.png"

# Preprocess the image
test_img = preprocess_image(test_img_path)

# Predict the gesture
prediction = loaded_model.predict(test_img)

# Decode the predicted class (assuming you used LabelEncoder during training)
predicted_class = np.argmax(prediction, axis=1)

print(f'Predicted class: {predicted_class}')



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
Predicted class: [9]
